In [1]:

import string
import pandas as pd 
from tqdm.autonotebook import tqdm

from utils.RagPipeline import Ragpipeline

from utils.utils import normalize_path

C:\Users\kim_h\AppData\Local\Temp\ipykernel_12208\3463511841.py:3: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
# 제출용 샘플 파일 로드
submit_df = pd.read_csv("../open/sample_submission.csv")

df = pd.read_csv('../open/test.csv')
base_directory = '../open' # Your Base Directory

df.head()

,SAMPLE_ID,Source,Source_path,Question
0,TEST_000,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?
1,TEST_001,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?
2,TEST_002,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,중소벤처기업부의 혁신창업사업화자금(융자) 사업근거는 어떤 법률에 근거하고 있나요?
3,TEST_003,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,2010년에 신규 지원된 혁신창업사업화자금은 무엇인가요?
4,TEST_004,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,혁신창업사업화자금 중 2020년에 신규 지원된 자금은 무엇인가요?


In [3]:
source_list = list(df['Source'].unique())
# 알파벳 리스트
alphabet = list(string.ascii_uppercase)

# 딕셔너리로 파일명과 알파벳 매핑
file_mapping = {alphabet[i]: source_list[i] for i in range(len(source_list))}

reversed_file_mapping = {value: key for key, value in file_mapping.items()}

file_mapping

{'A': '중소벤처기업부_혁신창업사업화자금(융자)',
 'B': '보건복지부_부모급여(영아수당) 지원',
 'C': '보건복지부_노인장기요양보험 사업운영',
 'D': '산업통상자원부_에너지바우처',
 'E': '국토교통부_행복주택출자',
 'F': '「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》',
 'G': '「FIS 이슈 & 포커스」 23-2호 《핵심재정사업 성과관리》',
 'H': '「FIS 이슈&포커스」 22-2호 《재정성과관리제도》',
 'I': '「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》'}

In [4]:
# 대략 40분 걸림 
from config import config 

for i, source in enumerate(source_list):
    
    db_path = f'data/{reversed_file_mapping[source]}'
    pipeline = Ragpipeline(db_path, config)
    
    target_df = df[df['Source'] == source]
    
    for _, row in tqdm(target_df.iterrows(), total=len(target_df), desc=f"Answering Questions ({i+1}/{len(source_list)})"):
        # 소스 문자열 정규화
        source = normalize_path(row['Source'])
        # 질문 
        question = row['Question']
        
        # 대답
        full_response = pipeline.answer_generation(question)
        first_line_response = full_response.split("\n")[0]
        
        # 결과 저장
        submit_df.loc[submit_df['SAMPLE_ID'] == row['SAMPLE_ID'], 'Answer'] = first_line_response
        

Answering Questions (9/9): 100%|██████████| 20/20 [29:25<00:00, 88.30s/it]


In [5]:
submit_df

,SAMPLE_ID,Answer
0,TEST_000,"2022년 혁신창업사업화자금(융자의) 예산은 2,300,000백만원입니다."
1,TEST_001,중소벤처기업부의 혁신창업사업화자금(융자)의 사업목적은 기술력과 사업성이 우수하고 미...
2,TEST_002,"중소기업진흥에 관한 법률 제66조, 제67조, 제74조 및 중소기업창업지원법 제35..."
3,TEST_003,2010년에 신규 지원된 혁신창업사업화자금에 대한 정보는 없습니다.
4,TEST_004,"미래기술육성자금, 고성장촉진자금이 신규 지원되었습니다."
...,...,...
93,TEST_093,"공적보증채무는 재정정책 및 분석에서 중요하며, 보증인이 기타 공공부문과 민간부문 단..."
94,TEST_094,미래사회보장급여에 대한 순의무란 계약상의 부채가 아닌 미래 발생 비용으로 여겨지는 ...
95,TEST_095,국가회계기준과 지방자치단체 회계기준에서 우발부채에 대한 용어 및 회계처리가 다릅니다...
96,TEST_096,우발부채는 미래의 다양한 의제의무를 포괄하는 부채를 의미한다. 우발부채 관리가 중요...


In [6]:
import os 
save_path = './results'
os.makedirs(save_path, exist_ok=True)

In [7]:
# 0.6428 
save_file_name = '실험_0820_best_setting_1_intfloat_multilingual_PyPDFLoader.csv'
# 

In [8]:
submit_df.to_csv(f"./{save_path}/{save_file_name}", encoding='UTF-8-sig', index=False)

## 연속 실험

In [4]:
import os 
save_path = './results'
os.makedirs(save_path, exist_ok=True)

In [5]:

def experiments(config):

    for i, source in enumerate(source_list):
        root_dir = config['save_data_path']
        db_path = f'{root_dir}/{reversed_file_mapping[source]}'
        pipeline = Ragpipeline(db_path, config)
        
        target_df = df[df['Source'] == source]
        
        for _, row in tqdm(target_df.iterrows(), total=len(target_df), desc=f"Answering Questions ({i+1}/{len(source_list)})"):
            # 소스 문자열 정규화
            source = normalize_path(row['Source'])
            # 질문 
            question = row['Question']
            
            # 대답
            full_response = pipeline.answer_generation(question)
            first_line_response = full_response.split("\n")[0]
            
            # 결과 저장
            submit_df.loc[submit_df['SAMPLE_ID'] == row['SAMPLE_ID'], 'Answer'] = first_line_response
            
    return submit_df

In [6]:
pdf_loaders = ['pymupdf4llm', 'fitz', 'PyPDFLoader', 'PyMuPDFLoader', 'PDFPlumberLoader']

In [7]:
config = {
    "llm_predictor": {
        "model_name": "llama3.1-dacon-Q8:latest",   # llama3.1-dacon-Q8:latest,  llama3.1
        "temperature": 0
    },
    # "intfloat/multilingual-e5-small", intfloat/multilingual-e5-base, intfloat/multilingual-e5-large
    # BAAI/bge-m3
    "embed_model": "intfloat/multilingual-e5-large",  
    
    # pymupdf4llm, fitz, PyPDFLoader, PyMuPDFLoader, UnstructuredPDFLoader, PDFPlumberLoader, PyPDFDirectoryLoader
    "pdf_loader": "PyMuPDFLoader", 
    
    "text_split":{
        'chunk_size': 512,
        'chunk_overlap': 32
    },
    
    "save_data_path": './data_PyMuPDFLoader_512_32',
    
    "search_type": "mmr",           # "mmr"  similarity   similarity_score_threshold
    'search_kwargs_k': 3,           # 유사도 기반 상위 k개
    'search_kwargs_lambda': 0.5,    # 0~1, 0에 가까울수록 다양성, 1에 가까울수록 관련성
    'score_threshold': 0.4,         # 0~1, 쿼리 문장과 최소한 0.x 이상의 유사도를 가진 문서만
    
    'bm25_k' : 3,                   # 검색어 기반 상위 k개
    "ensemble_search_type": "mmr",  # 앙상블 서칭 타입
    "ensemble_weight": [0.5,0.5],   # bm25와 vector 가중치


}

In [8]:
print(save_path)
for pdf_loader in pdf_loaders:
    print(pdf_loader)
    config['pdf_loader'] = pdf_loader
    config['save_data_path'] = f'./data_{pdf_loader}_512_32'
    save_file_name = f'best_setting_intfloat_multilingual_{pdf_loader}.csv'
    
    submit_df = experiments(config)
    submit_df.to_csv(f"./{save_path}/{save_file_name}", encoding='UTF-8-sig', index=False)

./results
pymupdf4llm


Answering Questions (9/9): 100%|██████████| 20/20 [28:39<00:00, 85.97s/it]


fitz


Answering Questions (9/9): 100%|██████████| 20/20 [31:49<00:00, 95.47s/it] 


PyPDFLoader


Answering Questions (1/9):  33%|███▎      | 3/9 [04:10<08:16, 82.74s/it]